<a href="https://colab.research.google.com/github/Mehedi16009/Mali-GPU-Firmware-Attestation-Defense-system-Against-MOLE-Attack-/blob/main/FAARM_%E2%80%94_Firmware_Attestation_for_ARM_based_GPU_TEEs_Security_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

High-level overview (what I built FAARM — Firmware Attestation for ARM-based GPU TEEs Security system here)

1.Vendor side (offline)

 *   Create signing keypair (private key kept offline).
 *   Sign firmware + manifest → produce firmware.sig.


2.Device side (Colab simulation of EL3 / EL1 / GPU)


*   EL3 attestor (Python): verifies signature, checks manifest (version/anti-rollback), enforces lock, issues token.
*   EL1 simulation (Python): attempts to load firmware, requests secure session, tries TOCTOU overwrites (to test defenses).

*  Protected memory & MCU state are simulated (in-memory objects / files) so we can show behavior without hardware.






3.Test harness


*   Demonstrate: valid signed firmware → accepted and locked; tampered/unsigned firmware → rejected.
*   Demonstrate TOCTOU attempt to overwrite after token → blocked by lock.



4.Metrics


*   Measure verify time, lock time, and show negligible impact on runtime-critical path (verification is once per load).



### 📂 Files / Components We Will Create

| File / Component | Description |
|------------------|-------------|
| **Firmware_Attestation_MaliGPU_Prototype.ipynb** | The Colab notebook (main workspace). |
| **vendor_signer.py** | Offline signer that creates RSA/ECC keys and signs the firmware + manifest. |
| **firmware.bin** | Dummy firmware image (simulated binary). |
| **manifest.json** | Metadata file containing `version`, `mcu_id`, and `timestamp`. |
| **firmware.sig** | Signature produced by the vendor’s private key. |
| **el3_attestor.py** | EL3 attestation and verification logic (simulated secure monitor). |
| **el1_driver_sim.py** | Driver / kernel simulator representing the untrusted OS layer. |
| **device_state.json** | Simulated secure storage (stores public key, last_accepted_version, and lock flag). |
| **secure_task_sim.py** | Simulates submission of encrypted secure tasks and EL3-controlled placement. |

---

🧩 **Workflow Overview**
1. `vendor_signer.py` generates keys and signs the firmware.
2. `manifest.json` + `firmware.sig` accompany `firmware.bin`.
3. `el3_attestor.py` verifies the firmware signature and handles version locks.
4. `el1_driver_sim.py` mimics untrusted driver interaction.
5. `device_state.json` tracks secure state persistence.
6. `secure_task_sim.py` emulates secure task execution under EL3 supervision.

In [ ]:
# @title Cell A — Setup environment

In [1]:
# @title Colab: install dependencies
!apt-get update -qq
!apt-get install -y -qq openssl
!pip install cryptography


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../openssl_3.0.2-0ubuntu1.20_amd64.deb ...
Unpacking openssl (3.0.2-0ubuntu1.20) over (3.0.2-0ubuntu1.16) ...
Setting up openssl (3.0.2-0ubuntu1.20) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# @title Cell A — Setup environment

In [2]:
# @title Create a dummy firmware.bin and manifest.json
with open("firmware.bin", "wb") as f:
    f.write(b"This is a benign firmware image for FAARM prototype.\n" * 8)

manifest = {
    "version": 1,
    "mcu_id": "mali-mcu-v1",
    "timestamp": "2025-10-07T00:00:00Z",
    "notes": "Demo manifest for FAARM prototype"
}
import json
with open("manifest.json", "w") as f:
    json.dump(manifest, f, indent=2)

!ls -lh firmware.bin manifest.json


-rw-r--r-- 1 root root 424 Oct  7 10:53 firmware.bin
-rw-r--r-- 1 root root 132 Oct  7 10:53 manifest.json


In [ ]:
# @title Cell B — Create dummy firmware and manifest

In [3]:
# @title vendor_signer.py logic embedded here (run in Colab but treat as vendor-side)
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric.utils import encode_dss_signature
import json, base64

# Generate keypair (for demo). In production, do this offline in an HSM.
priv_key = ec.generate_private_key(ec.SECP256R1())
pub_key = priv_key.public_key()

# Save keys (vendor private would be kept offline; public is distributed to devices)
with open("vendor_priv.pem","wb") as f:
    f.write(priv_key.private_bytes(
        serialization.Encoding.PEM,
        serialization.PrivateFormat.PKCS8,
        serialization.NoEncryption()))
with open("vendor_pub.pem","wb") as f:
    f.write(pub_key.public_bytes(
        serialization.Encoding.PEM,
        serialization.PublicFormat.SubjectPublicKeyInfo))

# Sign firmware: hash of firmware + manifest JSON
def sign_firmware(priv, firmware_path="firmware.bin", manifest_path="manifest.json", sig_out="firmware.sig"):
    with open(firmware_path,"rb") as f: fw = f.read()
    with open(manifest_path,"rb") as f: man = f.read()
    # Compose the payload: firmware || manifest
    payload = fw + b"\n" + man
    signature = priv.sign(payload, ec.ECDSA(hashes.SHA256()))
    with open(sig_out,"wb") as f:
        f.write(signature)
    print("Signed", firmware_path, "->", sig_out)

sign_firmware(priv_key)
print("Vendor keys and firmware signature created.")


Signed firmware.bin -> firmware.sig
Vendor keys and firmware signature created.


In [ ]:
# @title Cell C — Vendor: generate keypair & sign firmware (offline role)

In [4]:
# @title Simulated secure storage for EL3: public key + last accepted version + firmware lock
import json
device_state = {
    "vendor_pub_pem": open("vendor_pub.pem","rb").read().decode(),
    "last_accepted_version": 0,
    "firmware_locked": False,
    "firmware_hash": None
}
with open("device_state.json","w") as f:
    json.dump(device_state, f, indent=2)
print("Device secure state initialized.")


Device secure state initialized.


In [ ]:
# @title Cell E — EL3 attestor: verify, anti-rollback, lock logic

In [5]:
# @title el3_attestor.py core functionality (run this cell as the EL3 service)
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import ec
import json, hashlib

def compute_payload_hash(firmware_path="firmware.bin", manifest_path="manifest.json"):
    with open(firmware_path,"rb") as f: fw = f.read()
    with open(manifest_path,"rb") as f: man = f.read()
    payload = fw + b"\n" + man
    h = hashlib.sha256(payload).hexdigest()
    return h, payload

def load_device_state():
    with open("device_state.json","r") as f:
        return json.load(f)

def save_device_state(state):
    with open("device_state.json","w") as f:
        json.dump(state, f, indent=2)

def verify_and_lock(firmware="firmware.bin", manifest="manifest.json", sig="firmware.sig"):
    state = load_device_state()
    pub_pem = state["vendor_pub_pem"].encode()
    pubkey = serialization.load_pem_public_key(pub_pem)
    h, payload = compute_payload_hash(firmware, manifest)
    sig_bytes = open(sig,"rb").read()
    try:
        pubkey.verify(sig_bytes, payload, ec.ECDSA(hashes.SHA256()))
    except Exception as e:
        return False, "Signature verification FAILED: " + str(e)

    # parse manifest and verify version
    with open(manifest,"r") as f:
        man = json.load(f)
    if man["version"] <= state["last_accepted_version"]:
        return False, "Anti-rollback: version too old"
    # pass: update device state (lock firmware)
    state["last_accepted_version"] = man["version"]
    state["firmware_locked"] = True
    state["firmware_hash"] = h
    save_device_state(state)
    return True, "Verified and locked"

# Example call when driver asks EL3 to verify+load:
ok, msg = verify_and_lock()
print(ok, msg)
print("Device state now:", json.load(open("device_state.json")))


True Verified and locked
Device state now: {'vendor_pub_pem': '-----BEGIN PUBLIC KEY-----\nMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAExuHpQeJUWkaErevt+W/QwzSLdQc8\ncmthPu9TP0jd5gd/U9xDUGyl5w4efRCyOeaFm33umF/D42rqkYh3YgSrFA==\n-----END PUBLIC KEY-----\n', 'last_accepted_version': 1, 'firmware_locked': True, 'firmware_hash': 'f1ad9a781903e0a6ca7f0197d5036ceb4d74ce173f000f3006e6cdb4bdf1d654'}


In [ ]:
# @title Cell E — EL3 attestor: verify, anti-rollback, lock logic

In [6]:
# @title el1_driver_sim.py behavior demonstration
import json, shutil, time
state = json.load(open("device_state.json"))

def driver_request_firmware_load():
    print("EL1: Requesting EL3 to verify and lock firmware...")
    # In practice driver hands package to EL3; here we call EL3 function directly
    from importlib import reload
    ok, msg = verify_and_lock()  # function from previous cell in this Colab context
    print("EL3 response:", ok, msg)
    return ok

def driver_try_overwrite_firmware(new_fw_content=b"malicious"):
    # driver attempts to overwrite firmware.bin (this simulates attacker)
    print("EL1: Attempting to overwrite firmware.bin ...")
    # simulate permission: in a vulnerable system, this would write; in our secure system we check device_state
    state = json.load(open("device_state.json"))
    if state["firmware_locked"]:
        print("EL3: Firmware is locked. Overwrite denied.")
        return False
    else:
        with open("firmware.bin","wb") as f:
            f.write(new_fw_content)
        print("EL1: Overwrite done (device was not locked).")
        return True

# Run test flow
# 1) Before verify: attempt overwrite -> should be allowed if unlocked
print("=== Test 1: attempt overwrite *before* EL3 verification ===")
# reset lock to False for test (simulate fresh device)
s = json.load(open("device_state.json"))
s["firmware_locked"] = False; s["last_accepted_version"] = 0; s["firmware_hash"]=None
json.dump(s, open("device_state.json","w"), indent=2)

# EL1 overwrites
driver_try_overwrite_firmware(b"malicious-firmware-before-verify\n")

# Now driver asks EL3 to verify/load (should fail because signature mismatches)
ok = driver_request_firmware_load()
print("After verify attempt, state:", json.load(open("device_state.json")))

# Reset firmware.bin to benign, re-sign, then verify and lock
with open("firmware.bin","wb") as f:
    f.write(b"This is a benign firmware image for FAARM prototype.\n" * 8)
# re-sign using vendor private in the demo env
sign_firmware(priv_key)
ok = driver_request_firmware_load()
print("After signing and verify, state:", json.load(open("device_state.json")))

# 2) Attempt TOCTOU overwrite *after* EL3 verified and locked
print("\n=== Test 2: attempt overwrite *after* EL3 verification (TOCTOU) ===")
driver_try_overwrite_firmware(b"malicious-after-verify\n")


=== Test 1: attempt overwrite *before* EL3 verification ===
EL1: Attempting to overwrite firmware.bin ...
EL1: Overwrite done (device was not locked).
EL1: Requesting EL3 to verify and lock firmware...
EL3 response: False Signature verification FAILED: 
After verify attempt, state: {'vendor_pub_pem': '-----BEGIN PUBLIC KEY-----\nMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAExuHpQeJUWkaErevt+W/QwzSLdQc8\ncmthPu9TP0jd5gd/U9xDUGyl5w4efRCyOeaFm33umF/D42rqkYh3YgSrFA==\n-----END PUBLIC KEY-----\n', 'last_accepted_version': 0, 'firmware_locked': False, 'firmware_hash': None}
Signed firmware.bin -> firmware.sig
EL1: Requesting EL3 to verify and lock firmware...
EL3 response: True Verified and locked
After signing and verify, state: {'vendor_pub_pem': '-----BEGIN PUBLIC KEY-----\nMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAExuHpQeJUWkaErevt+W/QwzSLdQc8\ncmthPu9TP0jd5gd/U9xDUGyl5w4efRCyOeaFm33umF/D42rqkYh3YgSrFA==\n-----END PUBLIC KEY-----\n', 'last_accepted_version': 1, 'firmware_locked': True, 'firmware_hash': '

False

In [ ]:
# @title Cell G — Simulate secure task submission (EL1 → EL3 decrypt & placement)

In [7]:
# @title secure_task_sim.py: application submits encrypted payload; EL3 will only decrypt/place if firmware locked & valid
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os, base64, hashlib

# simple symmetric encryption for demo (NOT production key management)
key = os.urandom(16)
iv = os.urandom(16)

def encrypt_payload(plaintext: bytes):
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    # pad to 16
    pad_len = 16 - (len(plaintext) % 16)
    plaintext_padded = plaintext + bytes([pad_len])*pad_len
    ct = encryptor.update(plaintext_padded) + encryptor.finalize()
    return ct

def el3_place_decrypted_payload(ct):
    state = json.load(open("device_state.json"))
    if not state["firmware_locked"]:
        return False, "EL3 refuses: firmware not attested/locked"
    # decrypt (EL3 has symmetric key for demo)
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    pt_padded = decryptor.update(ct) + decryptor.finalize()
    pad_len = pt_padded[-1]
    pt = pt_padded[:-pad_len]
    # "place" into protected memory (simulated file)
    with open("protected_memory.bin","wb") as f:
        f.write(pt)
    return True, "Placed plaintext into protected region"

# Demo
payload = b"SECRET_GPU_INPUT_DATA"
ct = encrypt_payload(payload)
ok, msg = el3_place_decrypted_payload(ct)
print("EL3 placement:", ok, msg)
if ok:
    print("Protected memory content:", open("protected_memory.bin","rb").read())


EL3 placement: True Placed plaintext into protected region
Protected memory content: b'SECRET_GPU_INPUT_DATA'


In [9]:
# @title Cell H — Timing & performance measurement
import time
# measure signature verification time approx
start = time.perf_counter()
ok, msg = verify_and_lock()
end = time.perf_counter()
print("verify_and_lock took {:.3f} ms; result: {}".format((end-start)*1000, ok))


verify_and_lock took 1.337 ms; result: False
